# Transmission lines

PowerGenome compiles information on the transmission constraints between model regions and the centroid-to-centroid distance between model regions.

In [1]:
from pathlib import Path

import pandas as pd
from powergenome.generators import load_ipm_shapefile
from powergenome.GenX import (
    network_max_reinforcement,
    add_cap_res_network
)
from powergenome.transmission import (
    agg_transmission_constraints,
)
from powergenome.util import (
    init_pudl_connection, 
    load_settings,
    check_settings
)
from powergenome.external_data import insert_user_tx_costs, load_user_tx_costs

## Import settings

In [3]:
pudl_engine, pudl_out, pg_engine = init_pudl_connection()
cwd = Path.cwd()

settings_path = (
    cwd.parent / "example_systems" / "CONUS-3-zone" / "settings"
)
settings = load_settings(settings_path)
settings["input_folder"] = settings_path.parent / settings["input_folder"]
check_settings(settings, pg_engine)

## Aggregate transmission constraints
EPA provides bulk transmission constraints between IPM regions. PowerGenome combines these constraints across aggregated model regions. The required settings parameters are:
- `model_regions`
- `region_aggregations`

In [4]:
settings["model_regions"]

['Eastern', 'Western', 'ERCOT']

In [5]:
settings["region_aggregations"]

{'Eastern': ['SPP_N',
  'SPP_NEBR',
  'SPP_SPS',
  'SPP_WAUE',
  'SPP_WEST',
  'FRCC',
  'S_SOU',
  'S_VACA',
  'MIS_AMSO',
  'MIS_AR',
  'MIS_D_MS',
  'MIS_IA',
  'MIS_IL',
  'MIS_INKY',
  'MIS_LA',
  'MIS_LMI',
  'MIS_MAPP',
  'MIS_MIDA',
  'MIS_MNWI',
  'MIS_MO',
  'MIS_WOTA',
  'MIS_WUMS',
  'S_C_KY',
  'S_C_TVA',
  'S_D_AECI',
  'PJM_AP',
  'PJM_ATSI',
  'PJM_COMD',
  'PJM_Dom',
  'PJM_EMAC',
  'PJM_PENE',
  'PJM_SMAC',
  'PJM_West',
  'PJM_WMAC',
  'NENG_CT',
  'NENG_ME',
  'NENGREST',
  'NY_Z_A',
  'NY_Z_B',
  'NY_Z_C&E',
  'NY_Z_D',
  'NY_Z_F',
  'NY_Z_G-I',
  'NY_Z_J',
  'NY_Z_K'],
 'ERCOT': ['ERC_PHDL', 'ERC_REST', 'ERC_WEST'],
 'Western': ['WEC_BANC',
  'WEC_CALN',
  'WEC_LADW',
  'WEC_SDGE',
  'WECC_AZ',
  'WECC_CO',
  'WECC_ID',
  'WECC_IID',
  'WECC_MT',
  'WECC_NM',
  'WECC_NNV',
  'WECC_PNW',
  'WECC_SCE',
  'WECC_SNV',
  'WECC_UT',
  'WECC_WY']}

In [6]:
transmission = agg_transmission_constraints(pg_engine=pg_engine, settings=settings)

No transmission value column (e.g. firm vs non-firm) was specified in the settings. The column 'firm_ttc_mw' will be used as a default. This is a change from previous versions of PG, where 'nonfirm_ttc_mw' was used. Firm transmission capacity is lower or equal to non-firm capacity.
The user transmission capacity table duplicates values from the database. Database values will be discarded in favor of user values.


In [7]:
transmission

,Network_Lines,z1,z2,z3,Line_Max_Flow_MW,Line_Min_Flow_MW,transmission_path_name
0,1,1,-1,0,1830.0,-1830.0,Eastern_to_Western
1,2,1,0,-1,820.0,-820.0,Eastern_to_ERCOT


## Line loss, reinforcement costs, max reinforcement

PowerGenome can calculate approximate transmission costs and line loss using the straight line centroid-to-centroid distance between regions but we recommend using pre-calculated values instead. These pre-calculated values use transmission line segments created with a least cost path method described in Appendix F of the paper [Land use trade-offs in decarbonization of electricity generation in the American West](https://www.sciencedirect.com/science/article/abs/pii/S2666278723000144) -- the method is also described in a [preprint version](https://arxiv.org/pdf/2211.05062.pdf).

The maximum reinforcement of transmission lines in a planning period is set with the parameters
- `tx_expansion_per_period`: The multiple of existing transmission capacity that can be built. A value of 1 would allow an existing line of 500MW to build another 500MW.
- `tx_expansion_mw_per_period`: The actual capacity that can be built per line. This parameter is useful to set a minimum amount of new transmission (e.g. a single 230kv line) when there is no existing transmission between two regions.

The larger value of the two parameters is used for each line.

In [8]:
settings["tx_expansion_per_period"]

10.0

In [9]:
settings["tx_expansion_mw_per_period"]

10000

In [10]:
user_tx_costs = load_user_tx_costs(
    settings["input_folder"]
                        / settings["user_transmission_costs"],
                        settings["model_regions"],
                        settings.get("target_usd_year"),
)
user_tx_costs

,start_region,dest_region,interconnect_cost_mw,interconnect_annuity_mw,line_loss_frac,start_id,dest_id,dollar_year,mw-km_per_mw,start_intraregion_cost_mw,dest_intraregion_cost_mw,start_intraregion_loss_frac,dest_intraregion_loss_frac,start_mw-km_per_mw,dest_mw-km_per_mw,total_interconnect_cost_mw,total_line_loss_frac,total_mw-km_per_mw,total_interconnect_annuity_mw,zone_1,zone_2,adjusted_dollar_year
0,Eastern,Western,630794.6187,30555.07645,0.045481,149221,202112,2018,797.906511,314827.3317,488518.1596,0.022696,0.031962,175.838766,310.273119,1671432.0,0.100139,1284.018397,80963.0,z1,z2,2022
1,Eastern,ERCOT,333984.6216,16177.88951,0.020748,149640,303537,2018,278.501046,314827.3317,271718.0809,0.022696,0.021446,175.838766,222.803249,1072840.0,0.064891,677.143062,51967.0,z1,z3,2022
2,Western,ERCOT,749213.1611,36291.15521,0.059687,300883,202112,2018,1047.143361,488518.1596,271718.0809,0.031962,0.021446,310.273119,222.803249,1759202.0,0.113096,1580.219730,85214.0,z2,z3,2022


In [11]:
transmission = insert_user_tx_costs(tx_df=transmission, user_costs=user_tx_costs)
transmission

,Network_Lines,z1,z2,z3,Line_Max_Flow_MW,Line_Min_Flow_MW,transmission_path_name,Line_Reinforcement_Cost_per_MWyr,Line_Reinforcement_Cost_per_MW,Line_Loss_Percentage
0,1,1,-1,0,1830.0,1830.0,Eastern_to_Western,80963.0,1671432.0,0.100139
1,2,1,0,-1,820.0,820.0,Eastern_to_ERCOT,51967.0,1072840.0,0.064891
2,3,0,1,-1,0.0,0.0,Western_to_ERCOT,85214.0,1759202.0,0.113096
